In [85]:
# Import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Logistic Regression
from sklearn.linear_model import LogisticRegression
import warnings

# 1) CPDs

## Fit logistic regression

In [86]:
elimination_scope = [("X1", ['X7', 'X8']),("X2", ['X4', 'X8']),("X3", ['X9', 'X8']),
                     ("X5", ['X9', 'X8']),("X4", ['X11', 'X8', 'X9']),("X6", ['X7', 'X10']),
                     ("X7", []),("X8", []),("X9", ['X10', 'X6']), ("X10", ['X7']),("X11", ['X8', 'X9'])]

df = pd.read_csv('dataset/50K_forward_simulation.csv')
print(df.head())

   X1  X2  X3  X4  X5  X6  X7  X8  X9  X10  X11
0   1   1   0   1   1   1   1   1   1    1    1
1   0   1   0   1   0   1   1   0   1    0    0
2   1   1   1   1   1   1   1   1   0    0    1
3   1   0   0   1   1   1   1   0   1    1    1
4   1   1   1   1   1   0   1   1   1    1    0


In [87]:
# Generate possible binary combinations in the same format as predict_proba requires
def generate_binary_combinations(n):
    # Python magic to generate all possible combinations of 0 and 1 in a string format
    str_combinations = [format(i, f"0{n}b") for i in range(2**n)]
    lst_combinations = []

    # Loops through all the string and converts them to a list of integers
    for elm in str_combinations:
        lst_elm = list(elm)
        for i in range(len(lst_elm)):
            lst_elm[i] = int(lst_elm[i])
        lst_combinations.append(lst_elm)
    return lst_combinations

print(generate_binary_combinations(3))


[[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]


In [88]:
inp_1 = ['X8', 'X9', 'X11']
out_1 = ['X4'] 

def log_reg_cpd(inp, out):
    # Splitting data into predictors (X) and outcome (y)
    X = df[inp]
    y = df[out]

    # Fit the logistic regression model
    warnings.filterwarnings("ignore")
    model = LogisticRegression()
    model.fit(X, y)

    # Predict probabilities for all combinations of 1's and 0's from parents
    combinations = generate_binary_combinations(len(inp))
    predictions = model.predict_proba(combinations)

    # Extracting the probabilities of child = 1
    probabilities = predictions[:, 1]

    # The two list combinations and probabilities are combined into a single list
    for i in range (len(combinations)):
        combinations[i].append(probabilities[i])

    #Generate columns
    inp.append('prob') 

    probabilities_df = pd.DataFrame(columns=inp, data=combinations)

    return probabilities_df

log_reg_cpd(inp_1, out_1)

,X8,X9,X11,prob
0,0,0,0,0.534918
1,0,0,1,0.740490
2,0,1,0,0.739466
3,0,1,1,0.875644
4,1,0,0,0.725492
5,1,0,1,0.867667
6,1,1,0,0.867055
7,1,1,1,0.941793
